In [ ]:
import pandas as pd
import glob
import os


files = glob.glob("*_raw.csv")

if not files:
    print("No *_raw.csv files found!")
else:
    print(f"Found {len(files)} movies: {files}")

dfs = []

for i, file in enumerate(files, start=1):  # movie_id starts from 1
    print(f"Processing ({i}/{len(files)}): {file}")
    df = pd.read_csv(file)


    raw_name = os.path.splitext(file)[0].replace("_raw", "")
    if 'movie_name' in df.columns and df['movie_name'].notna().any():
        movie_name = df['movie_name'].iloc[0].split(" Box Office")[0].strip()
    else:
        movie_name = raw_name

    df["movie"] = movie_name
    df["movie_id"] = i  # Unique ID: 1,2,3,...

    if 'day' in df.columns:
        df["primary_key"] = df["movie"] + " | " + df["day"].astype(str)
    else:
        df["primary_key"] = df["movie"] + " | Row_" + df.index.astype(str)


    desired_cols = [
        "movie_id", "movie", "day", "india_net", "change_percent",
        "karnataka", "aptg", "tamil_nadu", "kerala", "rest_of_india", "day_total",
        "budget", "verdict", "india_screens", "overseas_screens",
        "worldwide_screens", "release_date", "primary_key"
    ]
    df = df[[col for col in desired_cols if col in df.columns]]

    dfs.append(df)


combined_df = pd.concat(dfs, ignore_index=True)


combined_df.drop_duplicates(subset=["primary_key"], keep="first", inplace=True)

combined_df["day_num"] = combined_df["day"].str.extract(r'(\d+)').astype(float).fillna(999)
combined_df = combined_df.sort_values(["movie_id", "day_num"]).drop(columns=["day_num"], errors="ignore")

# Final cleanup
combined_df.reset_index(drop=True, inplace=True)

# ------------------- SAVE FINAL FILES -------------------
combined_df.to_csv("TFI_MASTER_DATASET.csv", index=False)
combined_df.to_excel("TFI_MASTER_DATASET.xlsx", index=False)

print("\nSUCCESS! ALL MOVIES COMBINED")
print(f"Total rows: {len(combined_df):,}")
print(f"Total movies: {combined_df['movie'].nunique()}")
print(f"Unique movie_ids: {sorted(combined_df['movie_id'].unique())}")
print("\nFiles saved:")
print("   TFI_MASTER_DATASET.csv")
print("   TFI_MASTER_DATASET.xlsx  ← Open in Power BI!")

Found 41 movies: ['Adipurush_raw.csv', 'agv_raw.csv', 'asvr_raw.csv', 'BAN_raw.csv', 'BN_raw.csv', 'BRO_raw.csv', 'D2_raw.csv', 'Dasara_raw.csv', 'DC_raw.CSV', 'devara_raw.csv', 'dhruva_raw.csv', 'dj_raw.csv', 'Eagle_raw.csv', 'F3_raw.csv', 'GG_raw.CSV', 'GK_raw.csv', 'HN_raw.CSV', 'JG_raw.csv', 'Kalki2898AD_raw.csv', 'kushi_raw.csv', 'liger_raw.csv', 'Maharshi_raw.csv', 'NPS_raw.csv', 'nsni_raw.csv', 'OG_raw.csv', 'RadheyShyam_raw.csv', 'Ravanasura_raw.csv', 'rgs_raw.csv', 'RRR_raw.csv', 'Saaho_raw.csv', 'saindav_raw.csv', 'Salaar_raw.csv', 'SK_raw.csv', 'SLN_raw.csv', 'SSR_raw.csv', 'SVP_raw.csv', 'TJ_raw.csv', 'TNR_raw.csv', 'VK_raw.csv', 'WAR2_raw.csv', 'wfl_raw.csv']
Processing (1/41): Adipurush_raw.csv
Processing (2/41): agv_raw.csv
Processing (3/41): asvr_raw.csv
Processing (4/41): BAN_raw.csv
Processing (5/41): BN_raw.csv
Processing (6/41): BRO_raw.csv
Processing (7/41): D2_raw.csv
Processing (8/41): Dasara_raw.csv
Processing (9/41): DC_raw.CSV
Processing (10/41): devara_raw.cs

In [2]:
import pandas as pd

df = pd.read_csv("TFI_MASTER_DATASET.csv")

# Mapping dictionary (add more when you add movies)
hero_map = {
    1: "Prabhas", 2: "Prabhas", 3: "Prabhas", 4: "Prabhas", 5: "Prabhas",
    6: "Jr NTR",   7: "Jr NTR",   8: "Jr NTR",   9: "Jr NTR",   10: "Jr NTR",
    11: "Ram Charan", 12: "Ram Charan", 13: "Ram Charan", 14: "Ram Charan", 15: "Ram Charan",
    # add next heroes here
}

df["hero"] = df["movie_id"].map(hero_map)

df.to_csv("TFI_MASTER_WITH_HERO.csv", index=False)
df.to_excel("TFI_MASTER_WITH_HERO.xlsx", index=False)

print("Done! Now every movie has its hero.")
print(df[["movie_id", "movie", "hero"]].drop_duplicates())

Done! Now every movie has its hero.
     movie_id                           movie        hero
0           1                       Adipurush     Prabhas
20          2                  Agnyaathavaasi     Prabhas
28          3  Aravinda Sametha Veera Raghava     Prabhas
33          4                 Bharat Ane Nenu     Prabhas
38          5                   Bheemla Nayak     Prabhas
55          6                             Bro      Jr NTR
69          7                      Drushyam 2      Jr NTR
74          8                          Dasara      Jr NTR
79          9                    Dear Comrade      Jr NTR
84         10                 Devara - Part 1      Jr NTR
125        11                          Dhruva  Ram Charan
131        12             Duvvada Jagannadham  Ram Charan
136        13                           Eagle  Ram Charan
141        14         F3: Fun and Frustration  Ram Charan
146        15                 Geetha Govindam  Ram Charan
151        16                   Gunt

In [1]:
import pandas as pd
import re

# Load your master file
df = pd.read_csv("Master.csv")
print(f"Loaded {len(df)} rows, {df['movie'].nunique()} movies")

# 1. Clean money columns
money_cols = ['india_net', 'day_total', 'karnataka', 'aptg',
              'tamil_nadu', 'kerala', 'rest_of_india']
for col in money_cols:
    df[col] = df[col].astype(str).str.extract(r'([\d\.]+)').astype(float)

# 2. Clean movie name
df['movie'] = df['movie'].str.split(' Box Office').str[0].str.strip()

# 3. Clean release_date
df['release_date'] = df['release_date'].astype(str).str.extract(r'(\d{1,2}\w{2} \w+ \d{4})')

# FULL HERO MAPPING — ALL YOUR CURRENT & FUTURE HEROES
hero_dict = {
    1: "Prabhas", 22: "Prabhas", 33: "Prabhas", 37: "Prabhas", 39: "Prabhas",

4: "Allu Arjun", 14: "Allu Arjun", 29: "Allu Arjun", 31: "Allu Arjun", 32: "Allu Arjun",

13: "Ram Charan", 17: "Ram Charan", 35: "Ram Charan", 47: "Ram Charan", 36: "Ram Charan/Jr.NTR",

3: "Jr NTR", 12: "Jr NTR", 21: "Jr NTR", 36: "Jr NTR/Ram Charan", 48: "Jr NTR",

5: "Mahesh Babu", 19: "Mahesh Babu", 26: "Mahesh Babu", 41: "Mahesh Babu", 43: "Mahesh Babu",

2: "Pawan Kalyan", 6: "Pawan Kalyan", 7: "Pawan Kalyan", 30: "Pawan Kalyan", 46: "Pawan Kalyan",

11: "Vijay Deverakonda", 18: "Vijay Deverakonda", 24: "Vijay Deverakonda",
25: "Vijay Deverakonda", 49: "Vijay Deverakonda",

10: "Nani", 20: "Nani", 42: "Nani", 44: "Nani", 40: "Nani",

9: "Ravi Teja", 15: "Ravi Teja", 23: "Ravi Teja", 34: "Ravi Teja", 45: "Ravi Teja",

8: "Venkatesh", 16: "Venkatesh", 27: "Venkatesh", 38: "Venkatesh"}
# Apply hero mapping
df['hero'] = df['movie_id'].map(hero_dict)

# Fill any missing heroes with "Unknown" (safety net)
df['hero'] = df['hero'].fillna("Unknown")

# Reorder columns
final_cols = ['movie_id','hero','movie','day','india_net','change_percent',
              'karnataka','aptg','tamil_nadu','kerala','rest_of_india','day_total',
              'budget','verdict','india_screens','overseas_screens',
              'worldwide_screens','release_date','primary_key']
df = df[final_cols]

# Save
df.to_csv("Master_cleaned.csv", index=False)
df.to_excel("Master_cleaned.xlsx", index=False)

print("\nCLEANING + HERO MAPPING DONE!")
print(f"Hero breakdown:\n{df['hero'].value_counts()}")
print("\nFirst 10 rows:")
print(df[['movie_id','hero','movie','day','india_net']].head(10))

Loaded 737 rows, 48 movies

CLEANING + HERO MAPPING DONE!
Hero breakdown:
hero
Prabhas              140
Allu Arjun           136
Mahesh Babu           99
Jr NTR                92
Pawan Kalyan          79
Ram Charan            55
Jr NTR/Ram Charan     42
Ravi Teja             25
Nani                  25
Vijay Deverakonda     25
Venkatesh             19
Name: count, dtype: int64

First 10 rows:
   movie_id     hero      movie                    day  india_net
0         1  Prabhas  Adipurush     Day 1 [1st Friday]      13.70
1         1  Prabhas  Adipurush           Week 1 Share      34.00
2         1  Prabhas  Adipurush   Day 2 [1st Saturday]       7.78
3         1  Prabhas  Adipurush           Week 2 Share       2.31
4         1  Prabhas  Adipurush     Day 3 [1st Sunday]       8.15
5         1  Prabhas  Adipurush     Day 4 [1st Monday]       2.15
6         1  Prabhas  Adipurush    Day 5 [1st Tuesday]       1.22
7         1  Prabhas  Adipurush  Day 6 [1st Wednesday]       0.62
8         

In [9]:
import pandas as pd
df = pd.read_csv('Master_cleaned.csv')
df.head()

,movie_id,hero,movie,day,india_net,change_percent,karnataka,aptg,tamil_nadu,kerala,rest_of_india,day_total,budget,verdict,india_screens,overseas_screens,worldwide_screens,release_date,primary_key
0,1,Prabhas,Adipurush,Day 1 [1st Friday],13.70,₹ 3.5 Cr,7.2,48.5,1.4,0.60,44.80,102.5,₹ 450 Cr * Approx,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Day 1 [1st Friday]
1,1,Prabhas,Adipurush,Week 1 Share,34.00,₹ 9.04 Cr,NaN,NaN,NaN,NaN,NaN,NaN,₹ 450 Cr * Approx,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Week 1 Share
2,1,Prabhas,Adipurush,Day 2 [1st Saturday],7.78,₹ 1.75 Cr,5.9,24.8,1.6,0.45,44.25,77.0,₹ 450 Cr * Approx,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Day 2 [1st Saturday]
3,1,Prabhas,Adipurush,Week 2 Share,2.31,₹ 0.89 Cr,NaN,NaN,NaN,NaN,NaN,NaN,₹ 450 Cr * Approx,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Week 2 Share
4,1,Prabhas,Adipurush,Day 3 [1st Sunday],8.15,₹ 2.25 Cr,5.2,28.5,1.6,0.45,45.75,81.5,₹ 450 Cr * Approx,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Day 3 [1st Sunday]


In [12]:
import pandas as pd
import numpy as np
import re

# -------------------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------------------
# Assuming 'Master_cleaned.csv' contains the raw, combined data (including currency/symbols)
df = pd.read_csv('Master_cleaned.csv')

print(f"Starting cleaning process on {len(df)} rows...")


# -------------------------------------------------------------------
# 2. CORE CLEANING FUNCTIONS
# -------------------------------------------------------------------

def clean_collection_value(series):
    """Removes currency, crore markers, percentage signs, and handles text junk."""
    if not pd.api.types.is_numeric_dtype(series):
        series = series.astype(str)
        # Remove currency symbols (₹, Cr, * Approx)
        series = series.str.replace('₹', '', regex=False)
        series = series.str.replace(r' Cr\*? Approx', '', regex=True)
        series = series.str.replace(' Cr', '', regex=False)
        series = series.str.replace('%', '', regex=False)
        series = series.str.replace('* Approx', '', regex=False) # For budget column
        
        # Replace explicit missing indicators with NaN
        series = series.replace({
            '-': np.nan, 
            'Not Found': np.nan, 
            'nan': np.nan,
            '': np.nan
        })
        
        # Complex data removal (e.g., [Te: 66.75 Cr;...])
        # We assume the main number (e.g., '13.7' in '13.7,₹ 3.5 Cr') is in india_net.
        # However, looking at the source data [1], some columns like 'india_net' already hold cleaned floats (13.7) while others contain junk.
        # To handle columns where collections might contain extra language breakdowns (like the old raw scrape output), we rely on forcing numeric conversion.

    # Attempt float conversion, coercing errors (junk data) to NaN
    return pd.to_numeric(series, errors='coerce')


# -------------------------------------------------------------------
# 3. APPLY NUMERICAL CLEANING
# -------------------------------------------------------------------

# Columns that should contain numeric collection or percentage values, potentially mixed with text/symbols:
COLLECTION_COLS = [
    'india_net', 'change_percent', 'karnataka', 'aptg', 'tamil_nadu', 
    'kerala', 'rest_of_india', 'day_total', 'budget'
]
# Ensure these columns exist before attempting to clean them
COLLECTION_COLS = [col for col in COLLECTION_COLS if col in df.columns]


for col in COLLECTION_COLS:
    # Overwrite the original column with the cleaned numeric values
    df[col] = clean_collection_value(df[col])


# -------------------------------------------------------------------
# 4. CLEANING NON-COLLECTION COLUMNS (Screens, Dates)
# -------------------------------------------------------------------

# Clean screens data (Extract the number from "India: 7000")
SCREEN_COLS = ['india_screens', 'overseas_screens', 'worldwide_screens']
for col in SCREEN_COLS:
    if col in df.columns:
        # Extracts digits only from strings like "India: 7000" or "Worldwide total: 10000"
        df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float, errors='coerce')
        df[col] = df[col].replace({'Not Found': np.nan}) # Handle missing values


# Clean release_date (Source format: "16th June 2023")
if 'release_date' in df.columns:
    # Extract the full date string, which is necessary for proper date parsing later
    # Example: '16th June 2023' [1]
    df['release_date'] = df['release_date'].astype(str).str.extract(r'(\d{1,2}\w{2} \w+ \d{4})')
    df['release_date'] = df['release_date'].fillna(df['release_date'].str.extract(r'(\d{1,2}\w+ \w+ \d{4})')).fillna(np.nan) # Catch variations
    # Convert to datetime object (better for analysis)
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')


# Clean movie column (Remove external junk often appended in titles)
if 'movie' in df.columns:
    df['movie'] = df['movie'].astype(str).str.split(' Box Office').str.str.strip()


# -------------------------------------------------------------------
# 5. KEEP IDENTIFIERS CLEAN
# -------------------------------------------------------------------

# Ensure categorical/text/identifier columns remain as strings/objects:
# 'movie_id' (should be int), 'hero', 'movie', 'day', 'verdict', 'primary_key'
if 'movie_id' in df.columns:
    df['movie_id'] = pd.to_numeric(df['movie_id'], errors='coerce').astype('Int64', errors='ignore') # Ensure correct integer type


# -------------------------------------------------------------------
# 6. SAVE CLEANED DATA
# -------------------------------------------------------------------

df.to_csv('Master_cleaned_V2.csv', index=False)
df.to_excel('Master_cleaned_V2.xlsx', index=False)
print("\nCleaned and saved to 'Master_cleaned_V2.csv' and 'Master_cleaned_V2.xlsx'!")
print(df[COLLECTION_COLS + ['release_date', 'hero', 'india_screens']].head())

Starting cleaning process on 737 rows...


ValueError: Expected value of kwarg 'errors' to be one of ['raise', 'ignore']. Supplied value is 'coerce'

In [1]:
import pandas as pd
import numpy as np
import re

# Load file
df = pd.read_csv("Master_cleaned.csv")

# -----------------------------
# 1. Strip spaces in all string columns
# -----------------------------
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

# -----------------------------
# 2. Remove unwanted characters inside numeric columns
# Example: "1 234", "12,300", "₹ 50000", "10.0%", "abc123"
# -----------------------------
def clean_numeric(x):
    if pd.isnull(x):
        return np.nan
    x = str(x)
    x = x.replace(" ", "")       # remove spaces
    x = re.sub(r"[^0-9.\-]", "", x)  # keep only numbers, dot, minus
    return x if x != "" else np.nan

for col in df.columns:
    # detect possible numeric columns
    if df[col].dtype == 'object':
        df[col] = df[col].apply(clean_numeric)

# -----------------------------
# 3. Convert cleaned numeric columns to proper dtype
# -----------------------------
for col in df.columns:
    # try converting to float
    try:
        df[col] = pd.to_numeric(df[col])
    except:
        pass

# -----------------------------
# 4. Replace empty strings with NaN
# -----------------------------
df.replace("", np.nan, inplace=True)

# -----------------------------
# 5. Optional: drop duplicates
# -----------------------------
df.drop_duplicates(inplace=True)

# -----------------------------
# 6. OPTIONAL: Fill missing values (customizable)
# -----------------------------
# df.fillna({"column_name": 0}, inplace=True)

# Save final cleaned file
df.to_csv("Master_final_cleaned.csv", index=False)

print("Data cleaning completed and saved as Master_final_cleaned.csv")


Data cleaning completed and saved as Master_final_cleaned.csv


In [2]:
import pandas as pd

# Load file
df = pd.read_csv("Master_cleaned.csv")
print(f"Loaded {len(df)} rows, {df['movie'].nunique()} movies")

# 1. Clean numeric columns (remove ₹, Cr, %, extract numbers)
num_cols = ['india_net', 'day_total', 'karnataka', 'aptg', 'tamil_nadu', 'kerala', 'rest_of_india']
for col in num_cols:
    df[col] = df[col].astype(str).str.extract(r'([\d\.]+)').astype(float).fillna(0)  # Fill missing with 0

# 2. Clean change_percent (remove %, extract number)
df['change_percent'] = df['change_percent'].astype(str).str.extract(r'([\d\.]+)').astype(float).fillna(0)

# 3. Clean budget (extract number)
df['budget'] = df['budget'].astype(str).str.extract(r'([\d\.]+)').astype(float).fillna(0)

# 4. Clean verdict (replace "Not Found" with "Unknown")
df['verdict'] = df['verdict'].replace("Not Found", "Unknown")

# 5. Clean release_date (extract date only if junk)
df['release_date'] = df['release_date'].astype(str).str.extract(r'(\d{1,2}\w{2} \w+ \d{4})')

# 6. Reorder columns (same as yours)
final_cols = df.columns.tolist()
df = df[final_cols]

# 7. Save super clean versions
df.to_csv("Master_super_cleaned.csv", index=False)
df.to_excel("Master_super_cleaned.xlsx", index=False)

print("\nSUPER CLEANING DONE!")
print("Saved → Master_super_cleaned.csv & .xlsx")
print("\nFirst 10 rows:")
print(df[['movie_id', 'hero', 'movie', 'day', 'india_net']].head(10))

Loaded 737 rows, 48 movies

SUPER CLEANING DONE!
Saved → Master_super_cleaned.csv & .xlsx

First 10 rows:
   movie_id     hero      movie                    day  india_net
0         1  Prabhas  Adipurush     Day 1 [1st Friday]      13.70
1         1  Prabhas  Adipurush           Week 1 Share      34.00
2         1  Prabhas  Adipurush   Day 2 [1st Saturday]       7.78
3         1  Prabhas  Adipurush           Week 2 Share       2.31
4         1  Prabhas  Adipurush     Day 3 [1st Sunday]       8.15
5         1  Prabhas  Adipurush     Day 4 [1st Monday]       2.15
6         1  Prabhas  Adipurush    Day 5 [1st Tuesday]       1.22
7         1  Prabhas  Adipurush  Day 6 [1st Wednesday]       0.62
8         1  Prabhas  Adipurush   Day 7 [1st Thursday]       0.38
9         1  Prabhas  Adipurush     Day 8 [2nd Friday]       0.24


In [3]:
df.head()

,movie_id,hero,movie,day,india_net,change_percent,karnataka,aptg,tamil_nadu,kerala,rest_of_india,day_total,budget,verdict,india_screens,overseas_screens,worldwide_screens,release_date,primary_key
0,1,Prabhas,Adipurush,Day 1 [1st Friday],13.70,3.50,7.2,48.5,1.4,0.60,44.80,102.5,450.0,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Day 1 [1st Friday]
1,1,Prabhas,Adipurush,Week 1 Share,34.00,9.04,0.0,0.0,0.0,0.00,0.00,0.0,450.0,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Week 1 Share
2,1,Prabhas,Adipurush,Day 2 [1st Saturday],7.78,1.75,5.9,24.8,1.6,0.45,44.25,77.0,450.0,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Day 2 [1st Saturday]
3,1,Prabhas,Adipurush,Week 2 Share,2.31,0.89,0.0,0.0,0.0,0.00,0.00,0.0,450.0,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Week 2 Share
4,1,Prabhas,Adipurush,Day 3 [1st Sunday],8.15,2.25,5.2,28.5,1.6,0.45,45.75,81.5,450.0,Flop,India: 7000,Overseas: 3000,Worldwide total: 10000,16th June 2023,Adipurush | Day 3 [1st Sunday]


In [4]:
import pandas as pd

df = pd.read_csv("Master_super_cleaned.csv")

# Split hero column into multiple rows
df["hero"] = df["hero"].str.split("/")     # Convert "Jr NTR/Ram Charan" → ["Jr NTR", "Ram Charan"]
df = df.explode("hero")                    # Creates 2 rows from 1

# Remove spaces
df["hero"] = df["hero"].str.strip()

df.to_csv("Master_super_cleaned_hero_fixed.csv", index=False)

print("Done! Rows split successfully.")


Done! Rows split successfully.
